TODO: Set up an exclusive alpha map; so it has to choose between alphas for multi prompts on a single image. For example, a region that has "jean luc picard" cannot be the same as a region that has "emma watson". If we want we could relax or play with those constraints; but let's keep it simple first. We want to eventualy build a map of the whole image with all classes that way.

To do that, add loss penalizing common alpha (dot product between offending alpha masks)

If total alpha dips below some threshold, or some kinda statistic - it means it failed to find the thing

To combine masks, we can have another optimization that allows weighted averages of all previously found masks until we get one we like.

TODO: Average the results across multiple runs. What can we do with that? 

TODO: Add some kinda priors for shape; like superpixels or something. Instance segmentation can kinda do this, so can bilaterl blurs...

TODO: Multiple classes - use stabledifusion prompt subtraction

TODO: Get mean prompt across texts

Ways to think about it:
- What can we chip away while keeping the given prompt?

In [ ]:
import rp
import nerf.sd as sd
import numpy as np
import torch
import torch.nn as nn
from ryan.source.learnable_textures import LearnableTexturePackRaster,LearnableTexturePackFourier
from ryan.source.learnable_textures import LearnableImageRaster,LearnableImageFourier, LearnableImageRasterBilateral, LearnableImageFourierBilateral
import icecream
from IPython.display import clear_output
from ryan.bilateral_blur import BilateralProxyBlur
import timm
from torchvision.transforms.functional import normalize
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from timm.models.vision_transformer import vit_base_patch16_224_dino

In [ ]:
if 's' not in dir():
    s=sd.StableDiffusion('cuda:1',"CompVis/stable-diffusion-v1-4")
    # s=sd.StableDiffusion('cuda:1',"/raid/xiangli/Codes/VOC-model/motorbike-colorful-g-ext")
    # s=sd.StableDiffusion('cuda:1',"/raid/xiangli/Codes/VOC-model/dog-colorful-randp2")
device=s.device

In [ ]:
#SETTINGS
def make_learnable_image(height, width, num_channels, foreground=None):
    #Here we determine our image parametrization schema
    bilateral_blur =  BilateralProxyBlur(foreground,**bilateral_kwargs)
    return LearnableImageFourierBilateral(bilateral_blur,num_channels) #A neural neural image
    return LearnableImageRasterBilateral(bilateral_blur,num_channels) #A neural neural image
    return LearnableImageFourier(height,width,num_channels) #A neural neural image

In [ ]:
def get_mean_embedding(prompts:list):
    return torch.mean(
        torch.stack(
            [s.get_text_embeddings(prompt) for prompt in prompts]
        ),
        dim=0
    ).to(device)

class BaseLabel:
    def __init__(self, name:str, embedding:torch.Tensor):
        #Later on we might have more sophisticated embeddings, such as averaging multiple prompts
        #We also might have associated colors for visualization, or relations between labels
        self.name=name
        self.embedding=embedding
        
    def get_sample_image(self):
        output=s.embeddings_to_imgs(self.embedding)[0]
        assert rp.is_image(output)
        return output

    def __repr__(self):
        return '%s(name=%s)'%(type(self).__name__,self.name)
        
class SimpleLabel(BaseLabel):
    def __init__(self, name:str):
        super().__init__(name, s.get_text_embeddings(name).to(device))

class MeanLabel(BaseLabel):
    #Test: rp.display_image(rp.horizontally_concatenated_images(MeanLabel('Dogcat','dog','cat').get_sample_image() for _ in range(1)))
    def __init__(self, name:str, *prompts):
        prompts=rp.detuple(prompts)
        super().__init__(name, get_mean_embedding(prompts))
    

In [ ]:
output_folder='stable_diffusion_samples'

prompts='''
Jean Luc Picard
Beverly Crusher
Jean Luc Picard and Beverly Crusher
Harry Styles
Emma Watson
Emma Watson and Harry Styles
Arnold Schwarzenegger
Danny Devito
Arnold Schwarzenegger and Danny Devito
Harry Potter
Hermoine Granger
Harry Potter and Hermoine Granger
Chimchar
Turtwig
Piplup
Chimchar and Turtwig and Piplup
Creeper
Minecraft Creeper
Minecraft Steve
Minecraft Steve and Creeper
Rintaroi Okabe Anime Boy
Makise Kurisu Anime Girl
Rintaroi Okabe Anime Boy from Steins Gate
Makise Kurisu Anime Girl from Steins Gate
Makise Kurisu Anime Girl and Rintaroi Okabe Anime Boy
Makise Kurisu Anime Girl and Rintaroi Okabe Anime Boy from Steins Gate
'''.strip().splitlines()

while True:
    for prompt in prompts:
        file_name=prompt+('_%05i'%(rp.millis()%10000))+'.png'
        image=SimpleLabel(prompt).get_sample_image()
        out_path=rp.path_join(output_folder,file_name)

        from IPython.display import clear_output
        clear_output()
        print(out_path)
        rp.display_image(image)
        rp.save_image(image,rp.path_join(output_folder,file_name))
        print('saved!')